In [25]:
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup 
from time import sleep
from datetime import date

In [26]:
# Uses undetected_chromedriver to bypass cloudflare and get source from dynamic content
options = uc.ChromeOptions() 
options.headless = True 
driver = uc.Chrome(use_subprocess=True, options=options) 

In [27]:
# Dataframe
columns = ['Course Title', 'Course Headline',  'Rating', 'Reviews', 'Course Length', 'Lectures', 'Level', 'Instructor' ]
df=pd.DataFrame(columns=columns)

In [28]:
# Data extractor
def scrapeCourses(courses):
    rows = []
    for course in courses:
        row = []

        try:
            course_info = course.find('div',class_="course-card--main-content--2XqiY")
        except:
            course_info = None
            continue

        try:         
            course_title=course_info.find("div", class_="ud-sr-only").previousSibling.text
        except: course_title=None

        try:
            course_head=course_info.find("span",{'data-purpose':'seo-headline'}).text
        except:
            course_head = None

        try:
            course_rating=course_info.find("span",{'data-purpose':'seo-rating'}).text
        except:
            course_rating = None

        try:
            course_reviews=course_info.find("span",{'data-purpose':'seo-num-reviews'}).text
        except:
            course_reviews = None

        try:
            course_hours=course_info.find("span",{'data-purpose':'seo-content-info'}).text
        except:
            course_hours=None

        try: 
            course_lectures = course_info.find("span",{'data-purpose':'seo-num-lectures'}).text
        except: 
            course_lectures= None
        try:
            course_level = course_info.find("span",{'data-purpose':'seo-instructional-level'}).text
        except: course_level= None

        try:
            course_instructor=course_info.find("div",{'data-purpose':"safely-set-inner-html:course-card:visible-instructors"}).text
        except:
            course_instructor= None

        row = [course_title, course_head, course_rating, course_reviews, course_hours, course_lectures, course_level, course_instructor]
        rows.append(row)
        
    return rows
    

In [29]:
# Params
query = 'programming'
page_limit= 50 

In [30]:
# Driver
for page in range(1,page_limit+1):
    driver.get(f'https://www.udemy.com/courses/search/?p={page}&q={query}&src=ukw')
    #driver.get("https://www.udemy.com/topic/javascript/")
    sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    courses_list = soup.find('div', {'class': 'course-list--container--3zXPS'})
    courses = courses_list.find_all('div', {'class': 'course-card--container--1QM2W'})
    rows = scrapeCourses(courses)
    tempdf = pd.DataFrame(rows,columns=columns)
    df=pd.concat([df,tempdf], ignore_index=True)

driver.close()

In [31]:
df

,Course Title,Course Headline,Rating,Reviews,Course Length,Lectures,Level,Instructor
0,Pre-Programming: Everything you need to know b...,Increase your chance of success learning to co...,Rating: 4.7 out of 5,10514 reviews,6.5 total hours,87 lectures,Beginner,Evan Kimbrell
1,Learn Python Programming Masterclass,This Python For Beginners Course Teaches You T...,Rating: 4.6 out of 5,96163 reviews,72.5 total hours,534 lectures,All Levels,"Tim Buchalka, Jean-Paul Roberts, Tim Buchalka'..."
2,Beginning C++ Programming - From Beginner to B...,Obtain Modern C++ Object-Oriented <strong>Prog...,Rating: 4.6 out of 5,55535 reviews,46 total hours,305 lectures,All Levels,"Tim Buchalka's Learn Programming Academy, Dr. ..."
3,Java Programming Masterclass updated to Java 17,Learn Java In This Course And Become a Compute...,Rating: 4.5 out of 5,175214 reviews,101 total hours,529 lectures,All Levels,"Tim Buchalka, Tim Buchalka's Learn Programming..."
4,Automate the Boring Stuff with Python Programming,A practical <strong>programming</strong> cours...,Rating: 4.7 out of 5,104716 reviews,9.5 total hours,51 lectures,All Levels,Al Sweigart
...,...,...,...,...,...,...,...,...
1395,C Language + Algorithms + Data Structures = Power,Up and running with c <strong>Programming</str...,Rating: 3.5 out of 5,334 reviews,17 total hours,94 lectures,All Levels,Nidhal Abidi
1396,The Complete C# Programming + Visual Studio De...,Master the C# <strong>programming</strong> lan...,Rating: 3.9 out of 5,219 reviews,18.5 total hours,144 lectures,All Levels,Morteza Kordi
1397,Functional Programming with Ramda,Broaden your <strong>programming</strong> skil...,Rating: 3.6 out of 5,30 reviews,1.5 total hours,10 lectures,Intermediate,Bernard Mordan
1398,Data Programming with F#,Explore practical techniques to handle real-wo...,Rating: 4.3 out of 5,28 reviews,2.5 total hours,14 lectures,Intermediate,Packt Publishing


In [32]:
# Output
df.to_csv(f'output/udemy_courses_{query}_{date.today()}.csv')
df.to_json(f'output/udemy_courses_{query}_{date.today()}.json')